<a href="https://colab.research.google.com/github/kylawolski/GGIS527_Final_Project_Wolski/blob/main/GGIS527_FinalProject_KylaWolski_CookCounty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assessing Social Vulnerability and Flooding Impacts in Cook County Neighborhoods**

Kyla Wolski

GGIS 527 Final Project

Fall 2025

NOTE: A lot of the cells that created visualizations are commented out, as this notebook was too large to copy to github with all of the outputs included. The visualizations can be created by removing the #'s in front of the code lines. They can also be found in my presentation recording and project report.

In [1]:
!pip install mapclassify pysal geodatasets --quiet

In [2]:
# import required libraries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium import Choropleth
import geodatasets
import numpy as np
import seaborn as sns
from sklearn.preprocessing import robust_scale
from sklearn.metrics import calinski_harabasz_score
from sklearn.cluster import KMeans, AgglomerativeClustering
from pysal.lib import weights
from shapely.geometry import Point
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
# data collection

# flood hazard data from FEMA NFHL (national flood hazard layer) last updated June 2024
fema = gpd.read_file("S_FLD_HAZ_AR.shp")
#fema.columns

In [5]:
# census data from ACS 2022 5 year estimates, downloaded via Social Explorer
census_data = gpd.read_file("cook_census.csv")

In [6]:
# for being able to plot the census tract data on a map,
# used the TIGER/line shapefile for illinois census tracts 2023

tracts = gpd.read_file("tl_2023_17_tract.shp")
tracts = tracts[tracts["COUNTYFP"] == "031"]
# set the projection
tracts = tracts.to_crs(4326)

In [7]:
# data merging/cleanup

In [8]:
# for FEMA flood zones
fema_filtered = fema[[
    "FLD_ZONE",     # FEMA flood zone (A, AE, AO, AH, VE, X)
    "SFHA_TF",      # in 100 ye floodplain? (True/False)
    "STATIC_BFE",   # base flood elevation (water surface height during 1% flood)
    "DEPTH",        # flood depth for shallow zones (AO/AH)
    "STUDY_TYP",    # type of flood study (FIS = flood ins study, D= detailed, L=limited, NONS)
    "geometry"      # polygon
]]

# key for what each flood zone means:
# AE – 1% annual chance (100-yr); BFE determined
# A – 1% annual chance; no BFE
# AH / AO – Shallow flooding (ponding/sheet flow)
# VE – Coastal high-hazard
# X – Outside 1% flood zone (minimal risk)
# 0.2 PCT ANNUAL CHANCE – 500-yr flood zone

In [9]:
# for census data
# filtering for columns I want to use for assessing social vulnerability for flooding

# the columns I want:
# geo info, Population Density (Per Sq. Mile), % Total Population over 65, % under age five, % with less than high school edu,
# % below poverty level, % no health insurance coverage,% occupied housing units no vehicle available, % total population white alone
# Note: combine % Total Population: 65 to 74 Years, % Total Population: 75 to 84 Years, and % Total Population: 85 Years and Over
# and omit the first row

census_filtered = census_data[[
    "State (FIPS Code)",
    "County of current residence",
    "Census Tract",
    "Geographic Identifier",
    "FIPS",
    "Population Density (Per Sq. Mile)",
    "% Total Population: Under 5 Years",
    "% Total Population: 65 to 74 Years",
    "% Total Population: 75 to 84 Years",
    "% Total Population: 85 Years and Over",
    "% Total Population: White Alone",
    "% Population 25 Years and Over: Less than High School",
    "% Population Age 18 to 64 for Whom Poverty Status  Is Determined: Living in Poverty",
    "% Population Age 65 and Over for Whom Poverty  Status Is Determined: Living in Poverty",
    "% Occupied Housing Units: No Vehicle Available"
]]

census_filtered["% Total Population: 65 Years and Over"] = census_filtered["% Total Population: 65 to 74 Years"] + census_filtered["% Total Population: 75 to 84 Years"] + census_filtered["% Total Population: 85 Years and Over"]
census_filtered["% Population 18 Years and Over: Living in Poverty"] = census_filtered["% Population Age 18 to 64 for Whom Poverty Status  Is Determined: Living in Poverty"] + census_filtered["% Population Age 65 and Over for Whom Poverty  Status Is Determined: Living in Poverty"]
census_filtered = census_filtered.drop(0)
census_filtered = census_filtered.drop(columns=["% Total Population: 65 to 74 Years", "% Total Population: 75 to 84 Years", "% Total Population: 85 Years and Over", "% Population Age 18 to 64 for Whom Poverty Status  Is Determined: Living in Poverty", "% Population Age 65 and Over for Whom Poverty  Status Is Determined: Living in Poverty"])

In [10]:
# merge the census data to the tracts shapefile for plotting
merged = tracts.merge(census_filtered, left_on="GEOID", right_on="FIPS", how="left")

In [11]:
# Convert all columns except GEOID and geometry to numeric
# and give a value of zero if nan
for col in merged.columns:
    if col not in ["GEOID", 'geometry']:
        merged[col] = pd.to_numeric(merged[col], errors="coerce").fillna(0)

In [12]:
# initial data exploration

In [13]:
#fema_filtered.plot(column="FLD_ZONE", legend=True, categorical=True, figsize=(10, 10))
#plt.title("FEMA Flood Zones for Cook County")

In [14]:
#fema_filtered.plot(column="SFHA_TF", legend=True, categorical=True, figsize=(10, 10))
#plt.title("In 100 Year Flood True/False For Cook County")

In [15]:
#fema_filtered.plot(column="STATIC_BFE", legend=True, figsize=(10, 10))
#plt.title("Cook County base flood elevation (water surface height during 100 year flood)")

In [16]:
#fema_filtered = fema_filtered.to_crs(4326)
#ax = fema_filtered.plot(color="white", alpha=0.2)
#fema_filtered.plot(ax=ax, column="SFHA_TF", cmap="coolwarm", alpha=0.7, legend=True)
#merged.plot(ax=ax, column="Population Density (Per Sq. Mile)", cmap="Greens", alpha=0.7, legend=True)
#plt.xlim(-88.2, -87.5)
#plt.title("Cook County Population Density (Per Sq. Mile) and True/False in 100 Year Floodplain")

In [17]:
#m = folium.Map(location=[41.85, -87.65], zoom_start=10)

#Choropleth(
 #   geo_data=merged,
  #  data=merged,
   # columns=["GEOID", "% Occupied Housing Units: No Vehicle Available"],
    #key_on="feature.properties.GEOID",
    #fill_color="YlOrRd",
    #fill_opacity=0.7,
    #line_opacity=0.2,
    #legend_name="% Occupied Housing Units: No Vehicle Available",
#).add_to(m)

#m

In [18]:
# use clustering for at socially vulnerable and at risk of flooding areas

In [19]:
cluster_variables = [
    "% Population 18 Years and Over: Living in Poverty",
    "% Total Population: 65 Years and Over",
    "% Total Population: Under 5 Years",
    "% Population 25 Years and Over: Less than High School",
    "% Total Population: White Alone",
    "% Occupied Housing Units: No Vehicle Available",
    "Population Density (Per Sq. Mile)"
]

In [20]:
#merged[cluster_variables].describe()

In [21]:
# Data normalization with robust scaling
gdf_scaled = robust_scale(merged[cluster_variables])

# Convert the scaled array back into a DataFrame
gdf_scaled_df = pd.DataFrame(gdf_scaled, columns=cluster_variables)

# Display summary statistics
#gdf_scaled_df.describe()

In [22]:
# k-means

# Run K-Means clustering (3 clusters)
np.random.seed(0)
kmeans = KMeans(n_clusters=3)
k3cls = kmeans.fit(gdf_scaled)

# Store cluster labels and check cluster sizes
merged['k3cls'] = kmeans.labels_
merged.groupby('k3cls').size()

,0
k3cls,
0,1327
1,1
2,4


In [23]:
# Reindex by cluster ID and select clustering variables
#tidy_db = merged.set_index("k3cls")[cluster_variables]

# Convert to long format for easier plotting
#tidy_db = tidy_db.stack().reset_index()
#tidy_db = tidy_db.rename(columns={"level_1": "Attribute", 0: "Values"})

# Preview transformed data
#tidy_db.head()

# Create KDE plots by cluster for each attribute
#facets = sns.FacetGrid(
#    data=tidy_db, col="Attribute", hue="k3cls", sharey=False,
#    sharex=False, aspect=2, col_wrap=3
#)
#_ = facets.map(sns.kdeplot, "Values", fill=True, warn_singular=False).add_legend()

In [24]:
# Map numeric cluster labels to descriptive names
cluster_names = {
    0: "Least vulnerable in this case",
    1: "Unknown",
    2: "Most vulnerable in this case"
}

# Create a new column with the descriptive cluster names
merged['cluster_name'] = merged['k3cls'].map(cluster_names)

In [25]:
# hierarchical clustering

# Perform hierarchical clustering (Ward linkage) with 3 clusters
np.random.seed(0)
model = AgglomerativeClustering(linkage="ward", n_clusters=3)
model.fit(gdf_scaled)

# Store cluster labels and check cluster sizes
merged["ward3"] = model.labels_
merged.groupby("ward3").size()

,0
ward3,
0,4
1,1327
2,1


In [26]:
# regionalization

# Generate spatial weights (Queen contiguity)
np.random.seed(0)
w = weights.Queen.from_dataframe(merged)

# Perform hierarchical clustering with spatial connectivity constraint
model = AgglomerativeClustering(linkage="ward", connectivity=w.sparse, n_clusters=3)
model.fit(gdf_scaled)

# Store cluster labels and check cluster sizes
merged["ward3wq"] = model.labels_
merged.groupby('ward3wq').size()

,0
ward3wq,
0,1330
1,1
2,1


In [27]:
# gepgraphical coherence
results = []
for cluster_type in ("k3cls", "ward3", "ward3wq"):
    # compute the region polygons using a dissolve
    regions = merged[[cluster_type, "geometry"]].dissolve(by=cluster_type)
    # compute the actual isoperimetric quotient for these regions
    ipqs = (
        regions.area * 4 * np.pi / (regions.boundary.length ** 2)
    )
    # cast to a dataframe
    result = ipqs.to_frame(cluster_type)
    results.append(result)
# stack the series together along columns
pd.concat(results, axis=1)

,k3cls,ward3,ward3wq
0,0.360965,0.143048,0.386450
1,0.755940,0.360965,0.755940
2,0.143048,0.755940,0.486401


In [28]:
# feature coherence

ch_scores = []
for cluster_type in ("k3cls", "ward3", "ward3wq"):
    # compute the CH score
    ch_score = calinski_harabasz_score(
        # using scaled variables
        robust_scale(merged[cluster_variables]),
        # using these labels
        merged[cluster_type],
    )
    # and append the cluster type with the CH score
    ch_scores.append((cluster_type, ch_score))

# re-arrange the scores into a dataframe for display
pd.DataFrame(
    ch_scores, columns=["cluster type", "CH score"]
).set_index("cluster type")

,CH score
cluster type,
k3cls,26041.149673
ward3,26041.149673
ward3wq,1750.128097


In [29]:
# merge the flood and acs data together
gdf = gpd.sjoin(merged, fema_filtered, how="left", predicate="intersects")

# only include areas designated in the 100 year floodplain
gdf = gdf[gdf["SFHA_TF"] == "T"]

In [30]:
# visualize k-means within the 100 year flooplain
#gdf.explore(column='k3cls', categorical=True, legend_kwds={'caption': 'Cluster'})

In [31]:
# modeling now to predict outcomes like damage severity from a flood event
# for full state of IL now (not enough data for just cook county)

In [32]:
# 1. load the NFIP claims dataset last updated nov 19 2025 https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2
# now using different years, however --> this could lead to bias an/or innacurate results
df_claims = gpd.read_file('FimaNfipClaims.csv')

In [33]:
# filter flood ins claims for IL and get total damage amount for each event
df_illinois_claims = df_claims[df_claims['state'] == 'IL']
df_illinois_claims['buildingDamageAmount'] = df_illinois_claims['buildingDamageAmount'].fillna(0)
df_illinois_claims['contentsDamageAmount'] = df_illinois_claims['contentsDamageAmount'].fillna(0)
df_illinois_claims['totalDamageAmount'] = df_illinois_claims['buildingDamageAmount'] + df_illinois_claims['contentsDamageAmount']

In [34]:
df_illinois_claims = df_illinois_claims[['totalDamageAmount', 'latitude', 'longitude']]

In [35]:
# for claims convert lat lon to points in order to use geometry and spatial join w census data

df_illinois_claims["totalDamageAmount"] = pd.to_numeric(df_illinois_claims["latitude"], errors="coerce")
df_illinois_claims["latitude"] = pd.to_numeric(df_illinois_claims["latitude"], errors="coerce")
df_illinois_claims["longitude"] = pd.to_numeric(df_illinois_claims["longitude"], errors="coerce")

# create geometry
df_illinois_claims["geometry"] = df_illinois_claims.apply(
    lambda row: Point(row["longitude"], row["latitude"]),
    axis=1
)

# convert to GeoDataFrame with correct CRS
df_illinois_claims = gpd.GeoDataFrame(
    df_illinois_claims,
    geometry="geometry",
    crs="EPSG:4326"   # claims are always in WGS84
)


In [36]:
# state census data (instead of just cook county like I did earlier)
il_census_data = gpd.read_file("il_census.csv")

In [37]:
# for census data
# filtering for columns I want to use for assessing social vulnerability for flooding

# the columns I want:
# geo info, Population Density (Per Sq. Mile), % Total Population over 65, % under age five, % with less than high school edu,
# % below poverty level, % no health insurance coverage,% occupied housing units no vehicle available, % total population white alone
# Note: combine % Total Population: 65 to 74 Years, % Total Population: 75 to 84 Years, and % Total Population: 85 Years and Over
# and omit the first row

il_census_filtered = il_census_data[[
    "State (FIPS Code)",
    "County of current residence",
    "Census Tract",
    "Geographic Identifier",
    "FIPS",
    "Population Density (Per Sq. Mile)",
    "% Total Population: Under 5 Years",
    "% Total Population: 65 to 74 Years",
    "% Total Population: 75 to 84 Years",
    "% Total Population: 85 Years and Over",
    "% Total Population: White Alone",
    "% Population 25 Years and Over: Less than High School",
    "% Population Age 18 to 64 for Whom Poverty Status  Is Determined: Living in Poverty",
    "% Population Age 65 and Over for Whom Poverty  Status Is Determined: Living in Poverty",
    "% Occupied Housing Units: No Vehicle Available"
]]

il_census_filtered["% Total Population: 65 Years and Over"] = il_census_filtered["% Total Population: 65 to 74 Years"] + il_census_filtered["% Total Population: 75 to 84 Years"] + il_census_filtered["% Total Population: 85 Years and Over"]
il_census_filtered["% Population 18 Years and Over: Living in Poverty"] = il_census_filtered["% Population Age 18 to 64 for Whom Poverty Status  Is Determined: Living in Poverty"] + il_census_filtered["% Population Age 65 and Over for Whom Poverty  Status Is Determined: Living in Poverty"]
il_census_filtered = il_census_filtered.drop(0)
il_census_filtered = il_census_filtered.drop(columns=["% Total Population: 65 to 74 Years", "% Total Population: 75 to 84 Years", "% Total Population: 85 Years and Over", "% Population Age 18 to 64 for Whom Poverty Status  Is Determined: Living in Poverty", "% Population Age 65 and Over for Whom Poverty  Status Is Determined: Living in Poverty"])

In [38]:
il = gpd.read_file("tl_2023_17_tract.shp")
# set the projection
il = il.to_crs(4326)

In [39]:
# merge the census data to the tracts shapefile for plotting
illinois = il.merge(il_census_filtered, left_on="GEOID", right_on="FIPS", how="left")

In [40]:
# Convert all columns except GEOID and geometry to numeric
# and give a value of zero if nan
for col in illinois.columns:
    if col not in ["GEOID", 'geometry']:
        illinois[col] = pd.to_numeric(illinois[col], errors="coerce").fillna(0)

In [41]:
joined = gpd.sjoin(
    df_illinois_claims,
    illinois,
    how="left",
    predicate="within"
)


In [42]:
# X = predictors
X = joined[[
    "% Population 18 Years and Over: Living in Poverty",
    "% Total Population: 65 Years and Over",
    "% Total Population: Under 5 Years",
    "% Population 25 Years and Over: Less than High School",
    "% Total Population: White Alone",
    "% Occupied Housing Units: No Vehicle Available",
    "Population Density (Per Sq. Mile)"
]].fillna(0)

# y = target
y = joined["totalDamageAmount"] # the damage amount is in whole dollars

In [43]:
# change nans to 0 (kept getting a lot of errors with this when trying to run the random forest model)
cols = joined.columns.drop("geometry")
joined[cols] = joined[cols].apply(pd.to_numeric, errors="coerce").fillna(0)
y = y.fillna(0)
y.isna().sum()

np.int64(0)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (18, 7)
Shape of X_test: (5, 7)
Shape of y_train: (18,)
Shape of y_test: (5,)


In [45]:
# Initialize the Random Forest Regressor
random_forest_model = RandomForestRegressor(random_state=42)

# Train the model
random_forest_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = random_forest_model.predict(X_test)

# Evaluate the model using RMSE
rmse_rf = root_mean_squared_error(y_test, y_pred_rf)

print(f"Random Forest - Root Mean Squared Error (RMSE): {rmse_rf}")

Random Forest - Root Mean Squared Error (RMSE): 0.7087792831497094


In [46]:
# Get feature importances from the trained Random Forest model
feature_importances_rf = random_forest_model.feature_importances_

# Create a pandas Series for better visualization
feature_importance_series_rf = pd.Series(feature_importances_rf, index=X_train.columns)

# Sort the features by importance in descending order
ranked_feature_importances_rf = feature_importance_series_rf.sort_values(ascending=False)

# Plot the ranked feature importances as a vertical bar chart
#plt.figure(figsize=(10, 6))
#plt.bar(ranked_feature_importances_rf.index, ranked_feature_importances_rf.values)
#plt.title('Feature Importances from Random Forest')
#plt.xlabel('Features')
#plt.ylabel('Importance Score')
#plt.xticks(rotation=90)
#plt.tight_layout()
#plt.show()